In [1]:
import os
import torch
# 打印当前工作目录
print("当前工作目录:", os.getcwd())
# 更改工作目录到指定路径
os.chdir('/root/autodl-tmp/RPG-DiffusionMaster-main')
# 再次打印当前工作目录以确认更改
print("更改后的工作目录:", os.getcwd())

当前工作目录: /root/autodl-tmp/RPG-DiffusionMaster-main
更改后的工作目录: /root/autodl-tmp/RPG-DiffusionMaster-main


In [18]:
from RegionalDiffusion_base import RegionalDiffusionPipeline
from RegionalDiffusion_xl import RegionalDiffusionXLPipeline
from diffusers.schedulers import KarrasDiffusionSchedulers,DPMSolverMultistepScheduler
from mllm import local_llm,GPT4
from transformers import MllamaForConditionalGeneration, PreTrainedTokenizerFast
import random

In [3]:
pipe = RegionalDiffusionXLPipeline.from_single_file("../mod/albedobaseXL_v20.safetensors", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config,use_karras_sigmas=True)

Some weights of the model checkpoint were not used when initializing CLIPTextModel: 
 ['text_model.embeddings.position_ids']
Some weights of the model checkpoint were not used when initializing CLIPTextModelWithProjection: 
 ['text_model.embeddings.position_ids']



开始init了！！！


In [4]:
pipe.enable_xformers_memory_efficient_attention()

In [5]:
prompt= 'a cat on the top of a sofa'

In [6]:
model_path='/root/autodl-tmp/llama3'

In [7]:
model_path='/root/autodl-tmp/llama3'

In [8]:
model_id=model_path
print('Using model:',model_id)

Using model: /root/autodl-tmp/llama3


In [9]:
#from transformers import LlamaForCausalLM, LlamaTokenizer
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_id)
print("tokenizer ok!!")

tokenizer ok!!


In [10]:
model = MllamaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)
print("model ok!!")

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

model ok!!


In [11]:
with open('template/template.txt', 'r') as f:
        template=f.readlines()
user_textprompt=f"Caption:{prompt} \n Let's think step by step:"
textprompt= f"{' '.join(template)} \n {user_textprompt}"
print(textprompt)

You are a master of composition who excels at extracting key objects and their attributes from input text and supplementing the original text with more detailed imagination, creating layouts that conform to human aesthetics. Your task is described as follows:
 
 Extract the key entities and their corresponding attributes from the input text, and determine how many regions should be splited.
 For each key object identified in the previous step, use precise spatial imagination to assign each object to a specific area within the image and start numbering from 0. The area refers to dividing the entire image into different regions for a general layout. Each key entities is assigned to a region. And for each entity in the region, give it a more detailed description based on the original text. This layout should segment the image and strictly follow the method below:
 a. Determine if the image needs to be divided into multiple rows (It should be noted that a single entity should not be split 

In [12]:
model_input = tokenizer(textprompt, return_tensors="pt").to("cuda")

In [13]:
#para_dict = GPT4(prompt,key='sk-UDZJFwAlNjn5bnQOOEJn8W7JXvm225kzvOXZARvVtuiQnPnR')
#para_dict = local_llm(prompt,model_path='/root/autodl-tmp/Llama2.13_1208') 
para_dict = local_llm(prompt,mod=model,tok=tokenizer,mod_input=model_input,txtpro=textprompt)

waiting for LLM response
output1-------------------------------------------------------
You are a master of composition who excels at extracting key objects and their attributes from input text and supplementing the original text with more detailed imagination, creating layouts that conform to human aesthetics. Your task is described as follows:
 
 Extract the key entities and their corresponding attributes from the input text, and determine how many regions should be splited.
 For each key object identified in the previous step, use precise spatial imagination to assign each object to a specific area within the image and start numbering from 0. The area refers to dividing the entire image into different regions for a general layout. Each key entities is assigned to a region. And for each entity in the region, give it a more detailed description based on the original text. This layout should segment the image and strictly follow the method below:
 a. Determine if the image needs to be 

In [14]:
split_ratio = para_dict['Final split ratio']    #获取字典中的Final split ratio:0.5,0.5,0.5;0.5,0.5,0.5
print(split_ratio)

1;1;1


In [15]:
regional_prompt = para_dict['Regional Prompt']      #获取字典中的Regional Prompt：每个区域的prompt
print(regional_prompt)

Lush green twintails cascade down, framing the girl's face with lively eyes and a subtle smile, accented by a few playful freckles BREAK A vibrant red blouse, featuring ruffled sleeves and a cinched waist, adorned with delicate pearl buttons, radiates elegance BREAK pleated blue skirt, knee-length, sways gracefully with each step, its fabric catching the light, paired with a slender white belt for a touch of sophistication.


In [16]:
negative_prompt = "NSFW,worst quality,low quality,normal quality,lowres,monochrome,grayscale,skin spots,acnes,skin blemishes,age spot,ugly,duplicate,morbid,mutilated,tranny,mutated hands,poorly drawn hands,blurry,bad anatomy,bad proportions,extra limbs,disfigured,missing arms,extra legs,fused fingers,too many fingers,unclear eyes,lowers,bad hands,missing fingers,extra digit,bad hands,missing fingers,extra arms and legs"    #没有negative

In [19]:
random_seed = random.randint(0, 100000)
images = pipe(
    prompt=regional_prompt,
    split_ratio=split_ratio, # The ratio of the regional prompt, the number of prompts is the same as the number of regions
    batch_size = 1, #batch size
    base_ratio = 0.5, # The ratio of the base prompt    
    base_prompt= prompt,       
    num_inference_steps=20, # sampling step
    height = 1024, 
    negative_prompt=negative_prompt, # negative prompt
    width = 1024, 
    seed = random_seed,# random seed
    guidance_scale = 7.0
).images[0]


开始call方法了!!!

开始keyconverter!!!

开始matrixdealer!!!
split_ratio 1;1;1
*******************************************************************************************************

r 0
c 0
[[[0, 1.0]], [[0, 1.0]], [[0, 1.0]]]
start:: 0 end:: 1.0 base:: 0.5 breaks:: 0
ooo 0 0.3333333333333333
row:: start:: 0 end 0.3333333333333333


r 1
c 0
[[[0, 1.0]], [[0, 1.0]], [[0, 1.0]]]
start:: 0 end:: 1.0 base:: 0.5 breaks:: 0
ooo 0.3333333333333333 0.6666666666666666
row:: start:: 0.3333333333333333 end 0.6666666666666666


r 2
c 0
[[[0, 1.0]], [[0, 1.0]], [[0, 1.0]]]
start:: 0 end:: 1.0 base:: 0.5 breaks:: 0
ooo 0.6666666666666666 1.0
row:: start:: 0.6666666666666666 end 1.0


********************************************************************************************************
self.split_ratio [<matrix.Row object at 0x7f46fb27b5e0>, <matrix.Row object at 0x7f46fb2e5070>, <matrix.Row object at 0x7f46fb38a670>]
self.base_ratio 0.5
**************开始后面的部分了**********************************************

  0%|          | 0/20 [00:00<?, ?it/s]

这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢
这里执行没有呢


In [20]:
random_number = str(random.randint(1, 9999)).zfill(6)
save_path = os.path.join("/root/autodl-tmp/T2I-CompBench-main/examples/samples", f"{prompt}_{random_number}.png")
images.save(save_path)
torch.cuda.empty_cache()

In [26]:
os.chdir('/root/autodl-tmp/T2I-CompBench-main/UniDet_eval')

In [27]:
print("更改后的工作目录:", os.getcwd())

更改后的工作目录: /root/autodl-tmp/T2I-CompBench-main/UniDet_eval


In [28]:
from experts.obj_detection.generate_dataset import Dataset, collate_fn
from experts.model_bank import load_expert_model
from accelerate import Accelerator
import argparse
import ruamel.yaml as yaml
from tqdm import tqdm
import spacy
import json
obj_label_map = torch.load('dataset/detection_features.pt')['labels']

In [29]:
def parse_args():
    parser = argparse.ArgumentParser(description="UniDet evaluation.")
    parser.add_argument(
        "--outpath",
        type=str,
        default="../examples/",
        help="Path to output score",
    )
    parser.add_argument(
        "--complex",
        type=bool,
        default=False,
        help="Prompt is simple structure or in complex category",
    )
    args = parser.parse_known_args()[0]
    return args

In [30]:
def get_mask_labels(depth, instance_boxes, instance_id):
    obj_masks = []
    obj_ids = []
    obj_boundingbox = []
    for i in range(len(instance_boxes)):
        is_duplicate = False
        mask = torch.zeros_like(depth)
        x1, y1, x2, y2 = instance_boxes[i][0].item(), instance_boxes[i][1].item(), \
                         instance_boxes[i][2].item(), instance_boxes[i][3].item()
        mask[int(y1):int(y2), int(x1):int(x2)] = 1
        if not is_duplicate:
            obj_masks.append(mask)
            obj_ids.append(instance_id[i])
            obj_boundingbox.append([x1, y1, x2, y2])

    instance_labels = {}
    for i in range(len(obj_ids)):
        instance_labels[i] = obj_ids[i].item()
    return obj_boundingbox, instance_labels

In [31]:
args = parse_args()

In [32]:
model1, transform = load_expert_model(task='obj_detection', ckpt="RS200")

/root/miniconda3/envs/RPG/lib/python3.9/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
Loading config experts/obj_detection/configs/Base-CRCNN-COCO.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


In [33]:
accelerator = Accelerator(mixed_precision='fp16')

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [34]:
config = yaml.load(open('configs/experts.yaml', 'r'), Loader=yaml.Loader)

In [53]:
outpath = args.outpath
data_path= outpath
save_path= f'{outpath}/labels'
batch_size = 64
dataset = Dataset(data_path,  transform)
data_loader = torch.utils.data.DataLoader(
    dataset=dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True,
    collate_fn=collate_fn,
)
model1, data_loader = accelerator.prepare(model1, data_loader)

In [54]:
 with torch.no_grad():
        result = []
        map_result = []
        for i, test_data in enumerate(tqdm(data_loader)):
            #print("-----------------------------------------------------------------------------------------")
            #print(i)
            #print("-----------------------------------------------------------------------------------------")
            test_pred = model1(test_data)
            for k in range(len(test_pred)):
                #print(test_pred[k])
                instance_boxes = test_pred[k]['instances'].get_fields()['pred_boxes'].tensor  # get the bbox of list
                instance_id = test_pred[k]['instances'].get_fields()['pred_classes']
                depth = test_data[k]['image'][0]

                # get score
                instance_score = test_pred[k]['instances'].get_fields()['scores']

                obj_bounding_box, obj_labels_dict = get_mask_labels(depth, instance_boxes, instance_id)
                #print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
                #print(obj_bounding_box)
                #print(obj_labels_dict)
                #print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
                obj = []
                for i in range(len(obj_bounding_box)):
                    obj_name = obj_label_map[obj_labels_dict[i]]
                    obj.append(obj_name)


                img_path_split = test_data[k]['image_path'].split('/')
                prompt = img_path_split[-1].split('_')[0] # get prompt from file names
                vocab_spatial = ['on side of', 'next to', 'near', 'on the left of', 'on the right of', 'on the bottom of', 'on the top of','on top of'] #locality words

                locality = None
                for word in vocab_spatial:
                    if word in prompt:
                        locality = word
                        break

                if (args.complex):
                    #for complex structure
                    nlp = spacy.load('en_core_web_sm')
                    # Define the sentence
                    sentence = prompt
                    # Process the sentence using spaCy
                    doc = nlp(sentence)
                    # Define the target prepositions
                    prepositions = ["on top of", "on bottom of", "on the left", "on the right",'next to','on side of','near']
                    # Extract objects before and after the prepositions
                    objects = []
                    for i in range(len(doc)):
                        if doc[i:i + 3].text in prepositions or doc[i:i + 2].text in prepositions or doc[i:i + 1].text in prepositions:
                            if doc[i:i + 3].text in prepositions:
                                k=3
                            elif doc[i:i + 2].text in prepositions:
                                k=2
                            elif doc[i:i + 1].text in prepositions:
                                k=1
                            preposition_phrase = doc[i:i + 3].text
                            for j in range(i - 1, -1, -1):
                                if doc[j].pos_ == 'NOUN':
                                    objects.append(doc[j].text)
                                    break
                                elif doc[j].pos_ == 'PROPN':
                                    objects.append(doc[j].text)
                                    break
                            flag=False
                            for j in range(i + k, len(doc)):
                                if doc[j].pos_ == 'NOUN':
                                    objects.append(doc[j].text)
                                    break
                                if(j==len(doc)-1):
                                    flag=True
                            if flag:
                                for j in range(i + k, len(doc)):
                                    if (j+1<len(doc)) and doc[j].pos_ == 'PROPN' and doc[j+1].pos_ != 'PROPN':
                                        objects.append(doc[j].text)
                                        break
                    if (len(objects)==2):
                        obj1=objects[0]
                        obj2=objects[1]
                    else:
                        obj1=None
                        obj2=None
                else:
                    print("普通场景")
                    #for simple structure
                    nlp = spacy.load("en_core_web_sm")
                    doc = nlp(prompt)
                    obj1= [token.text for token in doc if token.pos_=='NOUN'][0]
                    obj2= [token.text for token in doc if token.pos_=='NOUN'][-1]

                person = ['girl','boy','man','woman']
                if obj1 in person:
                    obj1 = "person"
                if obj2 in person:
                    obj2 = "person"
                print("*****检测到的物体如下:*****")
                print(obj)
                print("*****物体的分数如下*****")
                print(instance_score)
                print("*****原始语句*****")
                print(prompt)
                if obj1 in obj and obj2 in obj:
                    print("这哥们两个都有！")
                    obj1_pos = obj.index(obj1)
                    obj2_pos = obj.index(obj2)
                    obj1_bb = obj_bounding_box[obj1_pos]
                    obj2_bb = obj_bounding_box[obj2_pos]
                    box1, box2={},{}
                    box1["x_min"] = obj1_bb[0]
                    box1["y_min"] = obj1_bb[1]
                    box1["x_max"] = obj1_bb[2]
                    box1["y_max"] = obj1_bb[3]
                    box2["x_min"] = obj2_bb[0]
                    box2["y_min"] = obj2_bb[1]
                    box2["x_max"] = obj2_bb[2]
                    box2["y_max"] = obj2_bb[3]
                    print("*****第一个物体是*****")
                    print(obj1)
                    print("分数")
                    print(instance_score[obj1_pos].item())
                    print("*****第二个物体是*****")
                    print(obj2)
                    print("分数")
                    print(instance_score[obj2_pos].item())
                    print("*****获取到的空间信息*****")
                    print(locality)

                    score = 0.25 * instance_score[obj1_pos].item() + 0.25 * instance_score[obj2_pos].item()  # score = avg across two objects score
                    print("这哥们不算位置分",score)
                    score += determine_position(locality, box1, box2) / 2
                    print("这哥们算位置分",score)
                elif obj1 in obj:
                    print("这哥们只有obj1！")
                    obj1_pos = obj.index(obj1)
                    score = 0.25 * instance_score[obj1_pos].item()
                elif obj2 in obj:
                    print("这哥们只有obj2！")
                    obj2_pos = obj.index(obj2)
                    score = 0.25 * instance_score[obj2_pos].item()
                else:
                    score = 0
                print("=======================")
                print(score)
                print("=======================")
                if (score<0.5):
                    score=0
                print("=======================")
                print(score)
                print("=======================")

                image_dict = {}
                image_dict['question_id']=int(img_path_split[-1].split('_')[-1].split('.')[0])
                image_dict['answer'] = score
                result.append(image_dict)

                # add mapping
                map_dict = {}
                map_dict['image'] = img_path_split[-1]
                map_dict['question_id']=int(img_path_split[-1].split('_')[-1].split('.')[0])
                map_result.append(map_dict)
        im_save_path = os.path.join(save_path, 'annotation_obj_detection_2d')
        os.makedirs(im_save_path, exist_ok=True)

        with open(os.path.join(im_save_path, 'vqa_result.json'), 'w') as f:
            json.dump(result, f)
        print('vqa result saved in {}'.format(im_save_path))

        # avg score
        avg_score = 0
        print("--------------------------------------------------------------------------------------------")
        print(result)
        for i in range(len(result)):
            avg_score+=float(result[i]['answer'])
        with open(os.path.join(im_save_path, 'avg_score.txt'), 'w') as f:
            f.write('score avg:'+str(avg_score/len(result)))
        print("avg score:",avg_score/len(result))

        # save mapping
        with open(os.path.join(im_save_path, 'mapping.json'), 'w') as f:
            json.dump(map_result, f)

  0%|          | 0/1 [00:00<?, ?it/s]

普通场景
*****检测到的物体如下:*****
['cat', 'animal', 'carnivore', 'furniture', 'loveseat, couch', 'couch', 'studio couch', 'sofa bed', 'chair', 'crosswalk', 'pillow', 'bed', 'dog bed']
*****物体的分数如下*****
tensor([0.9937, 0.9751, 0.9590, 0.7671, 0.6987, 0.4734, 0.3555, 0.3262, 0.2917,
        0.1807, 0.1492, 0.1368, 0.1323], device='cuda:0', dtype=torch.float16)
*****原始语句*****
a cat on the top of a sofa
这哥们只有obj1！
0.2484130859375
0
普通场景
*****检测到的物体如下:*****
['cat', 'animal', 'carnivore', 'loveseat, couch', 'pillow', 'furniture', 'houseplant, potted plant', 'couch', 'sofa bed', 'studio couch', 'cat', 'animal', 'picture, frame', 'pillow', 'bed', 'carnivore', 'furniture', 'picture, frame', 'lynx', 'crosswalk']
*****物体的分数如下*****
tensor([0.9917, 0.9790, 0.9556, 0.8555, 0.8125, 0.7954, 0.7944, 0.5449, 0.4873,
        0.4111, 0.2783, 0.2114, 0.2050, 0.1953, 0.1776, 0.1727, 0.1635, 0.1167,
        0.1052, 0.1028], device='cuda:0', dtype=torch.float16)
*****原始语句*****
a cat on the top of a sofa
这哥们只有obj1！
0.2

100%|██████████| 1/1 [00:23<00:00, 23.66s/it]

*****检测到的物体如下:*****
['cat', 'animal', 'carnivore', 'loveseat, couch', 'furniture', 'couch', 'sofa bed', 'studio couch', 'lynx']
*****物体的分数如下*****
tensor([0.9961, 0.9868, 0.9780, 0.7417, 0.6782, 0.4626, 0.3176, 0.2903, 0.1798],
       device='cuda:0', dtype=torch.float16)
*****原始语句*****
a cat on the top of a sofa
这哥们只有obj1！
0.2490234375
0
vqa result saved in ../examples//labels/annotation_obj_detection_2d
--------------------------------------------------------------------------------------------
[{'question_id': 2, 'answer': 0}, {'question_id': 1139, 'answer': 0}, {'question_id': 1391, 'answer': 0}, {'question_id': 1655, 'answer': 0}, {'question_id': 1993, 'answer': 0}, {'question_id': 2468, 'answer': 0}, {'question_id': 2674, 'answer': 0}, {'question_id': 2690, 'answer': 0}, {'question_id': 3641, 'answer': 0}, {'question_id': 4085, 'answer': 0}, {'question_id': 4156, 'answer': 0}, {'question_id': 5331, 'answer': 0}, {'question_id': 5495, 'answer': 0}, {'question_id': 5801, 'answer': 0

In [55]:
torch.cuda.empty_cache()

In [31]:
file_path = '/root/autodl-tmp/T2I-CompBench-main/examples/labels/annotation_obj_detection_2d/avg_score.txt'

In [34]:
try:
    with open(file_path, 'r') as file:
        # 读取文件内容
        content = file.read()
        # 查找包含"score avg:"的行
        for line in content.splitlines():
            if "score avg:" in line:
                # 提取分数（假设分数紧跟在"score avg:"之后，且可能有空格分隔）
                score_str = line.split(":")[-1].strip()  # 去除冒号后的部分，并去除首尾空格
                # 尝试将分数转换为浮点数
                sco = float(score_str)
                break  # 找到分数后退出循环
except FileNotFoundError:
    print(f"文件 {file_path} 未找到。")
except ValueError:
    print("无法将分数转换为浮点数。")
except Exception as e:
    print(f"读取文件时发生错误: {e}")

In [35]:
# 打印结果
if sco is not None:
    print(f"分数是: {sco}")
else:
    print("未能从文件中获取分数。")

分数是: 0.0


In [37]:
counter = 0
 
while counter < 10:
    print("还不行")
    # 增加计数器
    counter += 1
    if sco != 0:
         break
    
    # 可选：在这里可以添加一些延迟（比如 sleep）来模拟实际工作中的等待时间
    time.sleep(1)  # 等待1秒
 
# 当计数器达到 10 时，自动退出循环
# 注意：由于我们没有改变 sco 的值，所以实际上是因为计数器达到了 10 才退出的循环
print("循环了10次，无论如何都退出了。")
 
# 如果你希望在循环退出后检查 sco 的值，可以在这里进行
# 但在这个例子中，sco 的值仍然是 0
print(f"最终 sco 的值是: {sco}")

还不行
还不行
还不行
还不行
还不行
还不行
还不行
还不行
还不行
还不行
循环了10次，无论如何都退出了。
最终 sco 的值是: 0.0


In [36]:
import time

In [1]:
import shutil

In [1]:
def determine_position(locality, box1, box2, iou_threshold=0.1,distance_threshold=150):
    # Calculate centers of bounding boxes
    box1_center = ((box1['x_min'] + box1['x_max']) / 2, (box1['y_min'] + box1['y_max']) / 2)
    box2_center = ((box2['x_min'] + box2['x_max']) / 2, (box2['y_min'] + box2['y_max']) / 2)

    # Calculate horizontal and vertical distances
    x_distance = box2_center[0] - box1_center[0]
    y_distance = box2_center[1] - box1_center[1]
    print("box1:",box1_center,"box2:",box2_center)
    print("x_distance:",x_distance,"y_distance:",y_distance)
    # Calculate IoU
    x_overlap = max(0, min(box1['x_max'], box2['x_max']) - max(box1['x_min'], box2['x_min']))
    y_overlap = max(0, min(box1['y_max'], box2['y_max']) - max(box1['y_min'], box2['y_min']))
    intersection = x_overlap * y_overlap
    box1_area = (box1['x_max'] - box1['x_min']) * (box1['y_max'] - box1['y_min'])
    box2_area = (box2['x_max'] - box2['x_min']) * (box2['y_max'] - box2['y_min'])
    union = box1_area + box2_area - intersection
    iou = intersection / union
    print("交并比:",iou)

    # Determine position based on distances and IoU and give a soft score
    score=0
    if locality in ['next to', 'on side of', 'near']:
        if (abs(x_distance)< distance_threshold or abs(y_distance)< distance_threshold):
            score=1
        else:
            score=distance_threshold/max(abs(x_distance),abs(y_distance))
    elif locality == 'on the right of':
        if x_distance < 0:
            if abs(x_distance) > abs(y_distance) and iou < iou_threshold:
                score=1
            elif abs(x_distance) > abs(y_distance) and iou >= iou_threshold:
                score=iou_threshold/iou
        else:
            score=0
    elif locality == 'on the left of':
        if x_distance > 0:
            if abs(x_distance) > abs(y_distance) and iou < iou_threshold:
                score=1
            elif abs(x_distance) > abs(y_distance) and iou >= iou_threshold:
                score=iou_threshold/iou
        else:
            score=0
    elif locality =='on the bottom of':
        if y_distance < 0:
            if abs(y_distance) > abs(x_distance) and iou < iou_threshold:
                score=1
            elif abs(y_distance) > abs(x_distance) and iou >= iou_threshold:
                score=iou_threshold/iou
    elif locality =='on the top of':
        if y_distance > 0:
            if abs(y_distance) > abs(x_distance) and iou < iou_threshold:
                score=1
            elif abs(y_distance) > abs(x_distance) and iou >= iou_threshold:
                score=iou_threshold/iou
    else:
        score=0
    print("位置得分:",score)
    return score